<a href="https://colab.research.google.com/github/aviax1/AE1/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install wandb

**dependencies**

In [19]:
# used snniped from https://github.com/L1aoXingyu/pytorch-beginner/
import torch
import wandb
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tensorflow.keras.datasets import mnist
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Process

**initial**

In [21]:
(xtrain,ytrain), (xtest,ytest) = mnist.load_data()
num_epochs=1000      #
batch_size = 64     #
image_size=784      #
hidden_size=128     #
lv_size = 64        # Latent Variable 
learning_rate=1e-4  #
cret = nn.MSELoss() # criterion

**build model**

In [22]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(image_size, hidden_size),   #nn.ReLU(True), nn.Linear(image_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True), nn.Linear(hidden_size, hidden_size), nn.ReLU(True), nn.Linear(hidden_size, lv_size))
        self.decoder = nn.Sequential(
            nn.Linear(lv_size, hidden_size),nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),nn.Linear(hidden_size, hidden_size),nn.ReLU(True), nn.Linear(hidden_size, image_size), nn.Tanh())

    def forward(self, x):
        return self.decoder(self.encoder(x))

**model setting**

In [23]:
model = autoencoder()
#wandb.watch(model)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

class DigitDataSet(Dataset):
  def __init__(self, dataset):
      self.dataset = dataset
      self.transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])

  def __len__(self):
      return len(self.dataset)

  def __getitem__(self, idx):
      if torch.is_tensor(idx):
          idx = idx.tolist()
      return self.transform( self.dataset[idx,:,:])

def images_row(dis_images,title,add_to_index=0, images_in_row=5):
  if( len(np.shape(dis_images)) == 2):
    dis_images=dis_images[0:images_in_row,:]
  else:
    dis_images=dis_images[0:images_in_row,0,:,:]
  for i in range(len(dis_images)):
    ax = plt.subplot(30, images_in_row, i+add_to_index + 1)
    plt.imshow(dis_images[i].reshape(28, 28))
    plt.title( title)
    plt.gray()

def visual_epoch(epoch_num,model,dataloader):
  for data in dataloader:
      input_imgs = data
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      output_imgs = model(imgs)
      images_row(input_imgs,"org ",5*epoch_num)
      images_row(output_imgs.detach().numpy(),"rec ",5*(epoch_num+1))

**train model by digit**

In [24]:
def train_by_digit(by_digit,model):
  wandb.init()
  print("*****\nstart traning Model for digit " +str(by_digit) +"\n")
  dataloader = DataLoader(DigitDataSet(xtrain[ytrain==by_digit]), batch_size=batch_size,shuffle=True, num_workers=4)
  visual_counter=0
  for epoch in range(num_epochs):
    run=1
    for data in dataloader:
      input_imgs = data
      imgs = Variable(input_imgs.view(input_imgs.size(0), -1))
      output_imgs = model(imgs)
      loss = cret(output_imgs, imgs)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      im=data[0,0,:,:].reshape(28,28)
      pred=model(imgs).detach().numpy()[0,:].reshape(28,28)
      if run:
        run=0
        wandb.log({"img": [wandb.Image(pred, caption="preidciton"),wandb.Image(im, caption="original")]})
    #if epoch%3==0 :
    #  visual_epoch(visual_counter,model,dataloader)
    print('epoch [{}/{}], loss:{:.4f}' .format(epoch + 1, num_epochs, loss.data))
    wandb.log({"loss": loss.data})

  torch.save(model.state_dict(), './ae_'+str(by_digit)+'.pth')
  print("\nfinish traning Model Number " +str(by_digit) +"\n")
  print("*****\n")

for by_digit in range(0,2):
  train_by_digit(by_digit,model)
  

*****
start traning Model for digit 0

epoch [1/300], loss:0.3206
epoch [2/300], loss:0.2436
epoch [3/300], loss:0.2521
epoch [4/300], loss:0.2767
epoch [5/300], loss:0.2072
epoch [6/300], loss:0.3124
epoch [7/300], loss:0.2808
epoch [8/300], loss:0.2289
epoch [9/300], loss:0.1701
epoch [10/300], loss:0.2099
epoch [11/300], loss:0.1423
epoch [12/300], loss:0.2521
epoch [13/300], loss:0.1972
epoch [14/300], loss:0.1916
epoch [15/300], loss:0.1391
epoch [16/300], loss:0.1088
epoch [17/300], loss:0.2010
epoch [18/300], loss:0.1842
epoch [19/300], loss:0.1277
epoch [20/300], loss:0.1906
epoch [21/300], loss:0.1504
epoch [22/300], loss:0.1603
epoch [23/300], loss:0.1099
epoch [24/300], loss:0.0894
epoch [25/300], loss:0.1308
epoch [26/300], loss:0.2216
epoch [27/300], loss:0.1317
epoch [28/300], loss:0.1759
epoch [29/300], loss:0.1411
epoch [30/300], loss:0.1605
epoch [31/300], loss:0.2325
epoch [32/300], loss:0.1702
epoch [33/300], loss:0.1402
epoch [34/300], loss:0.1561
epoch [35/300], lo

KeyboardInterrupt: ignored

Error in callback <function _init_jupyter.<locals>.cleanup at 0x7f61981a8a60> (for post_run_cell):


KeyboardInterrupt: ignored